<a href="https://colab.research.google.com/github/nikhilnair31/MIS284N-UD-YT-Shorts-Generator/blob/main/MIS284N_Unstructured_Data_Test_3B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

### Install and Import

In [4]:
!pip install praw openai elevenlabs replicate
!pip install opencv-python-headless moviepy
!pip install google-auth google-auth-oauthlib
!pip install --upgrade google-auth

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,010 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,182 kB]
Get:13 http:

In [5]:
import re
import os
import sys
import json
import time
import numpy as np
import pandas as pd
from datetime import datetime

from google.colab import drive
from IPython.display import Image, display

### Filter Warnings

In [6]:
import warnings

warnings.filterwarnings("ignore", category=UserWarning, message="It appears that you are using PRAW in an asynchronous environment.")
warnings.filterwarnings("ignore", category=DeprecationWarning)

### Load Drive and Env Vars

In [7]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# Specify the path to the shell script file
script_path = "/content/drive/MyDrive/Colab Notebooks/UD/env_vars.sh"

# Read the shell script file
with open(script_path) as file:
    script_content = file.readlines()

# Extract environment variables and assign them to os.environ
for line in script_content:
    if line.startswith("export "):
        key, value = line[len("export "):].strip().split("=")
        os.environ[key] = value.strip('"')

In [9]:
os.environ['REDDIT_USER_AGENT']

'Test UT'

# Reddit Shorts Generation

## Reddit Scrape

In [11]:
import praw

In [12]:
# Create a Reddit API instance
reddit = praw.Reddit(
    client_id=os.environ['REDDIT_CLIENT_ID'],
    client_secret=os.environ['REDDIT_CLIENT_SECRET'],
    user_agent=os.environ['REDDIT_USER_AGENT']
)

In [ ]:
total_media_to_retrieve = 35
results = []
params = {'limit': 7}  # Specify the number of items per request

while len(results) < total_media_to_retrieve:
    remaining_to_retrieve = total_media_to_retrieve - len(results)
    current_limit = min(remaining_to_retrieve, params['limit'])

    batch = list(reddit.subreddit("shortscarystories").top(limit=current_limit, params=params, time_filter="week"))

    if not batch:
        break

    results.extend(batch)

    params = {'limit': params['limit'], 'after': batch[-1].fullname}

In [14]:
# Create a dictionary to store post data
posts_dict = {}

# Iterate through the top posts
for i, post in enumerate(results):
    posts_dict[i] = {
        "Title": post.title,
        "Content": post.selftext,
        # "URL": post.url,
        "Timestamp": datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S UTC'),
        "Upvotes": post.ups,
        "Total Comments": post.num_comments,
    }

In [15]:
# Create a DataFrame from the dictionary
test_df = pd.DataFrame(posts_dict).T
# Convert "Timestamp" column to datetime
test_df["Timestamp"] = pd.to_datetime(test_df["Timestamp"])
# Check post content length
test_df["Post Length"] = test_df['Content'].apply(len)

test_df.head()

,Title,Content,Timestamp,Upvotes,Total Comments,Post Length
0,I deserved the divorce. But no one deserves wh...,"Alimony bleeds me dry every paycheck, but that...",2023-08-25 18:01:30+00:00,1065,62,2562
1,How to cheat Death,*Cheating Death is easy— People do it unconsci...,2023-08-28 03:13:08+00:00,1045,37,2474
2,My girlfriend says I’m becoming the man of her...,The day I bought DreamSync was the first time ...,2023-08-28 15:55:50+00:00,758,32,2685
3,I ran into my old roommate for the first time ...,"While walking down the street, I ran into my o...",2023-08-23 16:23:17+00:00,745,33,2758
4,“Lucky” Survivor,I was a “lucky” survivor.\n\nIt was a sunny da...,2023-08-26 03:48:54+00:00,571,28,2691


In [16]:
min_test_df = test_df.sort_values(by=['Post Length'], ascending=[True])[:7]
min_test_df

,Title,Content,Timestamp,Upvotes,Total Comments,Post Length
33,"I'm never telling anyone about my son's ""quirks""","He's different, but not worse. I love him with...",2023-08-29 15:24:36+00:00,90,6,610
27,License to Kill,"*One...*\n\nA methed up woman, screaming somet...",2023-08-25 19:27:04+00:00,112,9,1177
34,“The machines have a religion now”,"In a distant future, humanity’s insatiable que...",2023-08-25 03:08:31+00:00,78,7,1232
23,My Daughter's Brush With Death,My daughter Lisa used to be quite a happy and ...,2023-08-24 14:43:57+00:00,146,5,1246
29,Just a little while,Do you ever just wish you could disappear for ...,2023-08-26 14:22:10+00:00,105,13,1255
14,She’ll Only Play Once,"Her face is pale, and she is crying. Good. I l...",2023-08-27 16:10:52+00:00,227,13,1527
17,Blessed,I have the best husband! He works so hard for ...,2023-08-27 14:30:23+00:00,202,13,1909


## Seed Content

In [39]:
def preprocess_text(text):
    # Remove newlines and extra spaces
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()

    # Remove special characters and numbers
    # text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Remove asterisks (*) and backslashes (\)
    text = re.sub(r'[\*\\]', '', text)

    # Remove digits at the beginning of lines
    text = re.sub(r'^\d+\s+', '', text, flags=re.MULTILINE)

    return text

pd.set_option('display.max_colwidth', None)
seed_content = str(min_test_df['Content'].iloc[1])
preprocessed_content = preprocess_text(seed_content)
print(f'original text\n{seed_content}\n\nprocessed text\n{preprocessed_content}')

original text
*One...*

A methed up woman, screaming something about a demon?

*Two...*

I remember the first time I killed someone. The shock didn't last long, as another man ran at me, screaming incoherently. Never could tell the difference between late stage mental illness or developmental delays...

*Three...*

Third one in ten minutes? Must be heading into the bad part of town.

They call me in to put down...undesirables.

*Four...*

You know the types. Hard drug users, the severely mentally disabled, the ones with behavior problems, spoiled people who need to learn the word 'no' the hard way, the works.

*Five...*

Rich fucks don't want to get their hands dirty; politicians need the dregs of society to go away in order to get votes. Not that I care, I see it as a win-win-win. The more shots I fire off, the more I get paid. The more shots that I fire off in a short amount of time, means I'm about to eat good that week.

*Six...*
 
I's an easy job, once the disregard for human life

## Generating Title and Description

In [40]:
import openai

In [41]:
openai.api_key = os.environ['OPENAI_API_KEY']

In [42]:
def generate_title(prompt, script):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "system", "content": prompt},
                  {"role": "user", "content": script}],
        temperature=1,
        max_tokens=256,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )

    return response.choices[0].message['content']

In [46]:
shorts_title = generate_title("Generate a very short title for a YouTube Short based on the script. Make it similar to the examples below. Examples: - Scary things hidden in normal photos Part#28 - This is the scariest video on the internet... - Scary video😳 #shorts", preprocessed_content)
shorts_description = generate_title("Generate a very short description for a YouTube Short", preprocessed_content)

shorts_title = shorts_title.replace("\"","")
shorts_description = shorts_description.replace("\"","")

print(f'Title: {shorts_title}\nDescription: {shorts_description}\n')

Title: Chilling Encounters in the Night #shorts
Description: Intense thriller: A hired professional's chilling encounters in a dangerous job



In [47]:
folder_title = shorts_title.replace("\"","").split()[0]
folder_title

'Chilling'

## Image Prompts

In [48]:
def generate_image_prompts(gpt_type, prompt, script):
    response = openai.ChatCompletion.create(
        model=gpt_type,
        messages=[{"role": "system", "content": prompt},
                  {"role": "user", "content": script}],
        temperature=1,
        max_tokens=1024,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )

    image_prompts_str = response.choices[0].message['content']
    image_prompts_str = re.sub(r'[^\w\s]', '', image_prompts_str)
    image_prompts_str = re.sub(r'[^a-zA-Z\s]', '', image_prompts_str)
    image_prompts = image_prompts_str.lower().split('\n')
    image_prompts = list(map(lambda prompt: f"{prompt.strip()}", image_prompts))

    return image_prompts

In [50]:
gpt_type = "gpt-3.5-turbo"
gpt_prompt = """
  Generate 6 short one liner prompts for DALLE-2/Midjourney/StableDiffusion prompt image generation. The prompt I give will be the script for a video.
  Prompts should be very short but descriptive with no NSFW content.
"""
image_prompts = generate_image_prompts(gpt_type, gpt_prompt, preprocessed_content)
image_prompts

['a frantic woman desperately cries about a relentless demon',
 'memories of my first kill haunt me as another man charges towards me lost in his own world',
 'tension rises as i enter a dangerous neighborhood to deal with societal outcasts',
 'tasked with eliminating those deemed undesirables including drug users and behaviorally challenged individuals',
 'profitdriven politics collides with my morally detached mindset as i take down the unwanted',
 'with a numb conscience i find wealth and success in a job that requires ample ammunition',
 'in the midst of all the chaos its the innocent children that strike a chord within me',
 'eight nine ten the stakes escalate as my targets multiply kids unwittingly caught in the crossfire']

## Image Generation

In [51]:
import replicate

In [52]:
client = replicate.Client(api_token=os.environ["STABLEDIFFUSION_API_KEY"])

In [53]:
def generate_stablediffusion_image(image_prompt):
    output = client.run(
        "stability-ai/stable-diffusion:ac732df83cea7fff18b8472768c88ad041fa750ff7682a21affe81863cbe77e4",
        input = {
            "prompt": image_prompt,
            "height": 1024,
            "width": 576,
            "num_outputs": 1,
            "num_inference_steps": 50,
            "guidance_scale": 7.5,
            "scheduler": 'DPMSolverMultistep',
            "seed": 42
        }
    )
    return output[0]

In [55]:
generated_images = [generate_stablediffusion_image(prompt) for prompt in image_prompts]
generated_images

['https://pbxt.replicate.delivery/S77SOYl5n3IcDN6PwfKoMxpUGMX3VLPe0cLHBuPo9PIfRgeFB/out-0.png',
 'https://pbxt.replicate.delivery/SDSPcrORtbYfVK7pdDQUe0EMMkqBjeNwsxNHfJ6TPPgMkA9FB/out-0.png',
 'https://pbxt.replicate.delivery/IMRvEbTzrmLHGla9vS2jr0GAPPjXISWzLRg2uCz1CzfjEovIA/out-0.png',
 'https://pbxt.replicate.delivery/55tfN9XIEPXzDy81KEpqmi1nYlSp1t5kPSHc11dt2vglEovIA/out-0.png',
 'https://pbxt.replicate.delivery/xZQAjjWB2W5pCVnStGg179v9jvHOKihNpDjPoClXQc2TC0XE/out-0.png',
 'https://pbxt.replicate.delivery/T45DfhH2tet4VkVE6LVfQqVXO5XtnIxzk7TYCkoQk4BmSgeFB/out-0.png',
 'https://pbxt.replicate.delivery/F9qfpcfh2wsKKkmsBMcqQKUxvdP6CNef9Ot0hMjvWgqhlA9FB/out-0.png',
 'https://pbxt.replicate.delivery/Bztf2VmdXIxJOqj9fqVcD6RW9EXRpSwTYCffeVGRMVDiLB6LC/out-0.png']

In [56]:
import urllib.request

# Set the directory path to save the images
directory_path = f"/content/drive/MyDrive/Colab Notebooks/UD/{folder_title}/img"

# Create the directory if it does not exist
if not os.path.exists(directory_path):
    os.makedirs(directory_path)

# Save the images to the folder
for i, image_url in enumerate(generated_images):
    file_path = f"{directory_path}/image_{i+1}.jpg"
    urllib.request.urlretrieve(image_url, file_path)
    print(f"Image {i+1} saved successfully at {file_path}.")

Image 1 saved successfully at /content/drive/MyDrive/Colab Notebooks/UD/Chilling/img/image_1.jpg.
Image 2 saved successfully at /content/drive/MyDrive/Colab Notebooks/UD/Chilling/img/image_2.jpg.
Image 3 saved successfully at /content/drive/MyDrive/Colab Notebooks/UD/Chilling/img/image_3.jpg.
Image 4 saved successfully at /content/drive/MyDrive/Colab Notebooks/UD/Chilling/img/image_4.jpg.
Image 5 saved successfully at /content/drive/MyDrive/Colab Notebooks/UD/Chilling/img/image_5.jpg.
Image 6 saved successfully at /content/drive/MyDrive/Colab Notebooks/UD/Chilling/img/image_6.jpg.
Image 7 saved successfully at /content/drive/MyDrive/Colab Notebooks/UD/Chilling/img/image_7.jpg.
Image 8 saved successfully at /content/drive/MyDrive/Colab Notebooks/UD/Chilling/img/image_8.jpg.


## Audio Generation

In [57]:
from elevenlabs import set_api_key, generate, play

In [58]:
set_api_key(os.environ["ELEVENLABS_API_KEY"])

In [59]:
# Generate audio using 'elevenlabs' package
audio = generate(
    text=preprocessed_content,
    voice="Nicole",
    model="eleven_monolingual_v1"
)

# Set the directory path to save the images
directory_path = f"/content/drive/MyDrive/Colab Notebooks/UD/{folder_title}/aud"

# Create the directory if it does not exist
if not os.path.exists(directory_path):
    os.makedirs(directory_path)

audio_file_path = os.path.join(directory_path, f"audio_0.wav")

# Save the generated audio to the temporary directory
with open(audio_file_path, 'wb') as f:
    f.write(audio)

## Compositing

In [66]:
from moviepy.editor import ImageClip, concatenate_videoclips
from moviepy.editor import AudioFileClip

# Paths to images and audio
images_folder = f'/content/drive/MyDrive/Colab Notebooks/UD/{folder_title}/img/'
audio_file_path = f'/content/drive/MyDrive/Colab Notebooks/UD/{folder_title}/aud/audio_0.wav'

# Get image files from the folder. Sort image files by name. Create a list of image paths
image_files = [f for f in os.listdir(images_folder) if f.endswith('.jpg')]
image_files.sort()
image_paths = [os.path.join(images_folder, f) for f in image_files]

# Load audio clip
audio_clip = AudioFileClip(audio_file_path)

# Calculate duration per image
duration_per_image = audio_clip.duration / len(image_paths)

# Create video clips for each image
video_clips = []
for image_path in image_paths:
    img_clip = ImageClip(image_path, duration=duration_per_image).resize((1080, 1920))
    video_clips.append(img_clip)

# Concatenate video clips. Set audio for the concatenated video. Set the FPS for the final video
concatenated_clips = concatenate_videoclips(video_clips, method="compose")
final_clip = concatenated_clips.set_audio(audio_clip)
final_clip.fps = 24

# Set the directory path to save the images. Create the directory if it does not exist
directory_path = f"/content/drive/MyDrive/Colab Notebooks/UD/{folder_title}/output"
if not os.path.exists(directory_path):
    os.makedirs(directory_path)

# Export the video
formatted_timestamp = datetime.now().strftime('%Y%m%d%H%M%S')
output_video_path = directory_path+f'/output_{formatted_timestamp}_reddit.mp4'
final_clip.write_videofile(output_video_path, codec='libx264', threads=4)

Moviepy - Building video /content/drive/MyDrive/Colab Notebooks/UD/Chilling/output/output_20230830142738_reddit.mp4.
MoviePy - Writing audio in output_20230830142738_redditTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/Colab Notebooks/UD/Chilling/output/output_20230830142738_reddit.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/Colab Notebooks/UD/Chilling/output/output_20230830142738_reddit.mp4


In [67]:
import cv2
from moviepy.editor import VideoFileClip

def pipeline(frame, word):
    global curr_frame

    if curr_frame < frames_per_word and len(word) > 0:
        text = word[0]
    elif curr_frame >= frames_per_word:
        if len(word) > 1:
            text = word.pop(0)
        else:
            text = word[0]

        curr_frame = 0

    curr_frame += 1

    cv2.resize(frame, (1080, 1920))

    height, width, _ = frame.shape

    (text_width, text_height), _ = cv2.getTextSize(text, cv2.FONT_ITALIC, 4, 50)

    text_x = (width - text_width) // 2
    text_y = (height + text_height) // 2

    cv2.putText(frame, text, (text_x, text_y), cv2.FONT_ITALIC, 4, (0, 0, 0), 50, cv2.LINE_AA)
    cv2.putText(frame, text, (text_x, text_y), cv2.FONT_ITALIC, 4, (255, 255, 255), 10, cv2.LINE_AA)

    return frame

curr_frame = 0
words = [word.replace(",", "").replace(".", "") for word in preprocessed_content.upper().split()]
total_frames = int(cv2.VideoCapture(output_video_path).get(cv2.CAP_PROP_FRAME_COUNT))
frames_per_word = int(total_frames / len(words))

input_video_path = output_video_path
new_output_video_path = directory_path+f"/output_{formatted_timestamp}_caption_reddit.mp4"

video = VideoFileClip(input_video_path)
out_video = video.fl_image(lambda frame: pipeline(frame, words))
out_video.write_videofile(new_output_video_path, audio=True, preset='ultrafast')

Moviepy - Building video /content/drive/MyDrive/Colab Notebooks/UD/Chilling/output/output_20230830142738_caption_reddit.mp4.
MoviePy - Writing audio in output_20230830142738_caption_redditTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/Colab Notebooks/UD/Chilling/output/output_20230830142738_caption_reddit.mp4



t: 100%|█████████▉| 1870/1873 [01:20<00:00, 30.80it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/drive/MyDrive/Colab Notebooks/UD/Chilling/output/output_20230830142738_reddit.mp4, 6220800 bytes wanted but 0 bytes read,at frame 1872/1873, at time 78.00/78.03 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/Colab Notebooks/UD/Chilling/output/output_20230830142738_caption_reddit.mp4


## Uploading

In [68]:
import google.auth
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

In [69]:
# Load authorized user info from file
with open('/content/drive/MyDrive/Colab Notebooks/UD/token.json', 'r') as file:
    authorized_user_info = json.load(file)

creds = Credentials.from_authorized_user_info(authorized_user_info, ['https://www.googleapis.com/auth/youtube.upload'])
creds.refresh(Request())
credentials = creds

youtube = googleapiclient.discovery.build('youtube', 'v3', credentials=credentials)

In [70]:
# Define the function to upload the video file
def upload_short(video_title, video_description, video_file_path):
    request = youtube.videos().insert(
        part="snippet,status",
        body={
            "snippet": {
                "title": video_title,
                "description": video_description,
                "defaultLanguage": "en",
                "defaultAudioLanguage": "en"
            },
            "status": {
                "privacyStatus": "private",
                "madeForKids": False,
                "selfDeclaredMadeForKids": False,
                "embeddable": True,
                "license": "youtube",
                "publicStatsViewable": True
            },
        },
        media_body=googleapiclient.http.MediaFileUpload(video_file_path),
    )
    response = request.execute()
    return response

In [71]:
# Call the upload function with your desired parameters
response = upload_short(shorts_title, shorts_description, new_output_video_path)
response

{'kind': 'youtube#video',
 'etag': 'jZrr0kt2k8zt-V2CJFXXwlXtEa4',
 'id': 's4L17sKF5vc',
 'snippet': {'publishedAt': '2023-08-30T14:36:36Z',
  'channelId': 'UCWdV2EMmhOMenikQpuFdWog',
  'title': 'Chilling Encounters in the Night #shorts',
  'description': "Intense thriller: A hired professional's chilling encounters in a dangerous job",
  'thumbnails': {'default': {'url': 'https://i9.ytimg.com/vi/s4L17sKF5vc/default.jpg?sqp=CJSnvacG&rs=AOn4CLC_gOrZQf_U7QSeRUq0KqoJ3Vbk5w',
    'width': 120,
    'height': 90},
   'medium': {'url': 'https://i9.ytimg.com/vi/s4L17sKF5vc/mqdefault.jpg?sqp=CJSnvacG&rs=AOn4CLBGIHfea9BMoc3K3RK9_XNLsNdndA',
    'width': 320,
    'height': 180},
   'high': {'url': 'https://i9.ytimg.com/vi/s4L17sKF5vc/hqdefault.jpg?sqp=CJSnvacG&rs=AOn4CLDrWt10Yes2mCBPTYj11NtklOJdbQ',
    'width': 480,
    'height': 360}},
  'channelTitle': 'Nikhil Nair',
  'categoryId': '22',
  'liveBroadcastContent': 'none',
  'defaultLanguage': 'en',
  'localized': {'title': 'Chilling Encounters 